### 1. Calcular la duración media del alquiler (en días) para cada película:

```sql
SELECT f.title, AVG(DATEDIFF(r.return_date, r.rental_date)) AS avg_rental_duration_days
FROM rental r
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
WHERE r.return_date IS NOT NULL
GROUP BY f.title
ORDER BY avg_rental_duration_days DESC;
```

### 2. Calcular el importe medio de los pagos para cada miembro del personal:

```sql
SELECT staff_id, AVG(amount) AS avg_payment_amount
FROM payment
GROUP BY staff_id;
```

### 3. Calcular los ingresos totales para cada cliente, mostrando el total acumulado dentro del historial de alquileres de cada cliente:

```sql
SELECT 
    p.customer_id,
    p.payment_date,
    p.amount,
    SUM(p.amount) OVER (
        PARTITION BY p.customer_id 
        ORDER BY p.payment_date 
        ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
    ) AS cumulative_total
FROM payment p
ORDER BY p.customer_id, p.payment_date;
```

### 4. Determinar el cuartil para las tarifas de alquiler de las películas

```sql
SELECT title, rental_rate, NTILE(4) OVER (ORDER BY rental_rate) AS quartile
FROM film
ORDER BY rental_rate;
```

### 5. Determinar la primera y última fecha de alquiler para cada cliente:

```sql
SELECT
    customer_id,
    MIN(rental_date) OVER (PARTITION BY customer_id) AS first_rental_date,
    MAX(rental_date) OVER (PARTITION BY customer_id) AS last_rental_date
FROM
    rental;
```

### 6. Calcular el rango de los clientes basado en el número de sus alquileres:

```sql
SELECT 
    customer_id,
    COUNT(rental_id) AS total_rentals,
    RANK() OVER (ORDER BY COUNT(rental_id) DESC) AS rental_rank
FROM rental
GROUP BY customer_id
ORDER BY rental_rank;
```

### 7. Calcular el total acumulado de ingresos por día para la categoría de películas 'Familiar':

```sql
SELECT 
    DATE(p.payment_date) AS payment_day,
    SUM(p.amount) AS daily_total,
    SUM(SUM(p.amount)) OVER (ORDER BY DATE(p.payment_date)) AS cumulative_total
FROM payment p
JOIN rental r ON p.rental_id = r.rental_id
JOIN inventory i ON r.inventory_id = i.inventory_id
JOIN film f ON i.film_id = f.film_id
JOIN film_category fc ON f.film_id = fc.film_id
JOIN category c ON fc.category_id = c.category_id
WHERE c.name = 'Family'
GROUP BY DATE(p.payment_date)
ORDER BY payment_day;
```

### 8. Asignar un ID único a cada pago dentro del historial de pagos de cada cliente:

```sql
SELECT 
    p.customer_id,
    p.payment_id,
    p.payment_date,
    ROW_NUMBER() OVER (PARTITION BY p.customer_id ORDER BY p.payment_date) AS payment_sequence
FROM payment p
ORDER BY p.customer_id, payment_date;
```

### 9. Calcular la diferencia en días entre cada alquiler y el alquiler anterior para cada cliente:

```sql
SELECT
    customer_id,
    rental_id,
    rental_date,
    LAG(rental_date) OVER (PARTITION BY customer_id ORDER BY rental_date) AS previous_rental_date,
    DATEDIFF(rental_date, LAG(rental_date) OVER (PARTITION BY customer_id ORDER BY rental_date)) AS days_between_rentals
FROM
    rental
ORDER BY
    customer_id, rental_date;
```